In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline

In [2]:
houses = ['A', 'B', 'C']
xtype_dict = {'r':'raw', 'c':'change'}

In [3]:
def read_data(house, xtype):
    assert house in houses
    assert xtype in xtype_dict
    
    act_df = pd.read_csv("house{}_act.csv".format(house))
    sensor_df = pd.read_csv("house{}_sensor.csv".format(house))
    X = np.load("X_{}_house{}.npy".format(xtype_dict[xtype], house))
    Y = np.load("Y_house{}.npy".format(house))
    return act_df, sensor_df, X, Y

In [74]:
#read in data
house = 'C'
t = 'r'
act_df,sensor_df,X,Y = read_data(house, t)

In [75]:
#build 4 dict, act to idx, idx to act, sensor to idx, idx to sensor
all_act = list(set(act_df.label)) + [0] #all the act labels
a2i = dict(zip(all_act, range(len(all_act))))
i2a = dict(zip(range(len(all_act)), all_act))

all_sensor = list(set(sensor_df.label)) #all the sensor labels
i2s = dict(zip(range(len(all_sensor)), all_sensor))
s2i = dict(zip(all_sensor, range(len(all_sensor))))

In [76]:
print "# sensors: ", len(all_sensor)
print "# states/acts: ", len(all_act)
print "# timeframes: ", len(X)

# sensors:  21
# states/acts:  18
# timeframes:  26488


In [77]:
all_act

[1.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 22.0,
 28.0,
 0]

In [78]:
X[Y == all_act[0]]

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 1.,  0.,  0., ...,  1.,  1.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  1.,  1.,  0.],
       [ 0.,  0.,  1., ...,  1.,  1.,  0.],
       [ 0.,  0.,  1., ...,  1.,  1.,  0.]])

In [79]:
miu = np.zeros([len(all_act), len(all_sensor)])
for row in X[Y == all_act[0]]:
    miu[0] += row

In [80]:
sum(Y == all_act[0])

11961

In [81]:
miu[0]/11961

array([  4.24128417e-01,   2.34094139e-01,   1.02332581e-01,
         4.84909289e-03,   2.72803277e-01,   8.92901931e-01,
         9.99665580e-01,   3.64016387e-01,   1.83931109e-03,
         8.36050497e-05,   0.00000000e+00,   8.52437087e-01,
         3.83747178e-01,   6.49109606e-01,   2.68790235e-01,
         9.08870496e-01,   7.40824346e-01,   9.34704456e-01,
         8.45497868e-01,   9.79098738e-01,   1.29587827e-01])

## Naive Bayes

In [82]:
#each row represents sensors for a specific action i
#each column is sensor j
def calc_miu(X, Y, all_sensor=all_sensor, all_act=all_act):
    assert len(X)==len(Y)
    
    num_act = len(all_act)
    num_sensor = len(all_sensor)
    miu = np.zeros([num_act, num_sensor])
    for act in all_act:
        sensor = X[Y==act]
        for row in sensor:
            miu[a2i[act]] += row
        total = np.sum(Y==act)
        if total != 0:
            miu[a2i[act]] = miu[a2i[act]]/total
#         else:
#             miu[a2i[act]] = [1e-10]*len(miu[i])
    #normalize
#     for i,row in enumerate(miu):
#         total = np.sum(row)
#         if total != 0:
#             miu[i] = miu[i]/total
#         else:
#             miu[i] = [1e-10]*len(miu[i])
        
    return miu

In [83]:
def max_like(miu, X):
    num_time = len(X)
    num_act = len(miu)
    
    prob = np.zeros((num_time, num_act))
    for i in range(num_time):
        sensor = X[i]
        for j in range(num_act):
            act = miu[j]
            logp = 0
            for k,s in enumerate(sensor):
                if s == 1.:
                    logp += np.log(act[k])
                else:
                    logp += np.log(1-act[k])
            prob[i,j] = logp
    return prob

In [84]:
def mle(matrix):
    est = np.zeros(len(matrix))
    for i,row in enumerate(matrix):
        est[i] = np.argmax(row)
    return est

In [85]:
miu = calc_miu(X,Y)
likes = max_like(miu,X)
result = mle(likes) #note: result is index, not label yet

In [86]:
likes.shape

(26488, 18)

In [87]:
#convert index to label
res_label = np.array([i2a[e] for e in result])
for act in all_act:
    print "act label: {}, Y count:{}, estimate count:{}".format(act, sum(Y==act), sum(res_label==act))

act label: 1.0, Y count:11961, estimate count:4441
act label: 3.0, Y count:352, estimate count:373
act label: 4.0, Y count:78, estimate count:344
act label: 5.0, Y count:192, estimate count:1309
act label: 6.0, Y count:92, estimate count:1074
act label: 7.0, Y count:56, estimate count:497
act label: 9.0, Y count:62, estimate count:1328
act label: 10.0, Y count:7760, estimate count:2056
act label: 11.0, Y count:88, estimate count:629
act label: 12.0, Y count:9, estimate count:918
act label: 13.0, Y count:83, estimate count:1961
act label: 14.0, Y count:65, estimate count:2314
act label: 15.0, Y count:306, estimate count:728
act label: 16.0, Y count:8, estimate count:2074
act label: 17.0, Y count:22, estimate count:1984
act label: 22.0, Y count:40, estimate count:83
act label: 28.0, Y count:2457, estimate count:763
act label: 0, Y count:2857, estimate count:3612


# Evaluation

$$\text{Precision} = \frac{1}{N}\sum^N_i \frac{TP_i}{TI_i} $$
$$\text{Recall} = \frac{1}{N}\sum^N_i\frac{TP_i}{TT_i} $$
$$\text{F-Measure} = \frac{2 \cdot precision \cdot recal}{precision+recall} $$
$$\text{Accuracy} = \frac{\sum^N_i TP_i}{Total}$$

In [88]:
def precision(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TI = np.sum(pred_label==y)
        if TI != 0:
            res += (float(TP)/TI)
    return float(res)/N

def recall(pred_label, Y):
    all_label = list(set(Y))
    N = len(all_label)
    res = 0
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        TT = np.sum(Y==y)
        if TT != 0:
            res += float(TP)/TT
    return float(res)/N

def f_score(pred_label, Y):
    p = precision(pred_label, Y)
    r = recall(pred_label, Y)
    return 2*p*r/(p+r)

def accuracy(pred_label, Y):
    res = 0
    all_label = list(set(Y))
    for y in list(set(Y)):
        TP = np.sum(pred_label[Y==y]==y)
        res += TP
    return float(res)/len(Y)

In [89]:
precision(res_label, Y)

0.19438507246706388

In [90]:
recall(res_label, Y)

0.37346758693529

In [91]:
f_score(res_label, Y)

0.25568788927367625

In [92]:
accuracy(res_label, Y)

0.265289942615524

In [73]:
# #save miu
np.save("mu{}.npy".format(house), miu)

In [53]:
miu.shape

(25, 22)

In [93]:
pwd

u'/Users/yuhantang/AM207Project/correct/am207finalproject/data'